# Overcomplete retinal pyramid
## 1. Libraries & initialising parameters

In [5]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import os

import torch
torch.set_default_tensor_type('torch.DoubleTensor')

%load_ext watermark
%watermark -i -h -m -v -p numpy,torch,POLO  -r -g -b

ModuleNotFoundError: No module named 'watermark'

In [6]:
mode= 'bilinear'
width = 32
r_min = width / 8 #width/16 
r_max = width / 2 #7 * width/16
#base_levels = 1.61803
base_levels = 2
n_sublevel = 4 #cette fois on en met 4
n_azimuth = 2 #var 2->24
n_theta = 3 #var 2->24
n_phase = 2

## 2. Image after the ocular saccade (mask) #TODO names

In [7]:
img_orig = imread('../data/i05june05_static_street_boston_p1010764.jpeg')
ds = 1
if ds>1: img_orig = img_orig[::ds, ::ds]
img_orig = np.roll(img_orig, -162//ds, axis=1) # sliding gaze to the right by moving image to the left
img_orig = np.roll(img_orig, 32//ds, axis=0) # sliding gaze to the top by moving image to the bottom
from SLIP import Image
N_X, N_Y = img_orig.shape
pe = {'N_X': N_X, 'N_Y': N_Y, 'do_mask': True}

im = Image(pe)
img_orig -= img_orig.mean()
img_orig *= im.mask
img_tens = torch.Tensor(img_orig[None, None, ...])
print('Tensor shape=', img_tens.shape)

fig, ax = plt.subplots(figsize=(20,20))
ax.imshow(img_orig, cmap='gray')
ax.plot([img_orig.shape[1]/2], [img_orig.shape[0]/2], 'r+', ms=32);

NameError: name 'imread' is not defined

## 3. using torch to build up a Laplacian pyramid

In [ ]:
from torch.nn.functional import interpolate
img_down = img_tens
n_levels = 0
while max(img_down.shape[-2:]) > width :
    n_levels += 1
    print('Tensor shape=', img_down.shape, ', n_levels=', n_levels)
    #print(np.log(img_down.shape[-2:])/np.log(base_levels)-np.log(width)/np.log(base_levels))
    img_down = interpolate(img_down, scale_factor=1/base_levels, mode=mode)
n_levels += 1
print('Top tensor shape=', img_down.shape, ', Final n_levels=', n_levels)

## 4. Cropped pyramid

In [ ]:
def cropped_pyramid(img_tens, width=width, base_levels=base_levels, verbose=False):
    
    N_batch, _, N_X, N_Y = img_tens.shape
    n_levels = int(np.log(np.max((N_X, N_Y))/width)/np.log(base_levels)) + 1
    
    img_crop = torch.zeros((N_batch, n_levels, width, width))

    img_down = img_tens.clone()
    for i_level in range(n_levels-1):
        img_residual = img_down.clone()
        img_down = interpolate(img_down, scale_factor=1/base_levels, mode=mode)
        img_residual -= interpolate(img_down, size=img_residual.shape[-2:], mode=mode)

        if verbose: print('Tensor shape=', img_down.shape, ', shape=', img_residual.shape)
        h_res, w_res = img_residual.shape[-2:]

        img_crop[:, i_level, :, :] = img_residual[:, 0, 
                            (h_res//2-width//2):(h_res//2+width//2), 
                            (w_res//2-width//2):(w_res//2+width//2)]

    h_res, w_res = img_down.shape[-2:]
    img_crop[:, n_levels-1, 
             (width//2-h_res//2):(width//2+h_res//2), 
             (width//2-w_res//2):(width//2+w_res//2)] = img_down[:, 0, :, :]
    #if verbose: print('Top tensor shape=', img_down.shape, ', Final n_levels=', n_levels)
    return img_crop

## 5. Creating a set of filters

In [ ]:
from LogGabor import LogGabor
pe = {'N_X': width, 'N_Y': width, 'do_mask': False, 'base_levels':
          base_levels, 'n_theta': 24, 'B_sf': 0.6, 'B_theta': np.pi/12 ,
      'use_cache': True, 'figpath': 'results', 'edgefigpath':
          'results/edges', 'matpath': 'cache_dir', 'edgematpath':
          'cache_dir/edges', 'datapath': 'database/', 'ext': '.pdf', 'figsize':
          14.0, 'formats': ['pdf', 'png', 'jpg'], 'dpi': 450, 'verbose': 0}
lg = LogGabor(pe)

In [ ]:
def local_filter(azimuth, theta, phase, sf_0=.25, radius=width/4):

    x, y = lg.pe.N_X//2, lg.pe.N_Y//2 # center
    x += radius * np.cos(azimuth)
    y += radius * np.sin(azimuth)
    
    return lg.normalize(lg.invert(
        lg.loggabor(x, y, sf_0=sf_0, B_sf=lg.pe.B_sf, theta=theta, B_theta=lg.pe.B_theta) * np.exp(-1j * phase)))

K = local_filter(azimuth=0, theta=0, phase=0, radius=width/4)

In [ ]:
def get_K(width=width, n_sublevel = n_sublevel, n_azimuth = n_azimuth, n_theta = n_theta, 
          n_phase = n_phase, r_min = width/6, r_max = width/3, log_density_ratio = 2, verbose=False):
    K = np.zeros((width, width, n_sublevel, n_azimuth, n_theta, n_phase))
    for i_sublevel in range(n_sublevel):
        
        b = np.log(log_density_ratio)  / (r_max - r_min)
        a = (r_max - r_min) / (np.exp (b * (r_max - r_min)) - 1)
        c = r_min - a
        r_ref = r_min + i_sublevel * (r_max - r_min) / n_sublevel
        r_prim =  a * np.exp(b * (r_ref - r_min)) + c
        radius =  r_prim
        d_r_prim = a * b * np.exp(b * (r_ref - r_min))
        p_ref = 4 * width / 32 
        p_loc = p_ref * d_r_prim
        sf_0 = 1 / p_loc
        if verbose: print('r_ref', r_ref)
        if verbose: print('i_sublevel, sf_0, radius', i_sublevel, sf_0, radius)
        for i_azimuth in range(n_azimuth):
            for i_theta in range(n_theta):
                for i_phase in range(n_phase):
                    azimuth = (i_azimuth+i_sublevel/2)*2*np.pi/n_azimuth
                    theta = i_theta*np.pi/n_theta + azimuth
                    phase = i_phase*np.pi/n_phase
                    K[..., i_sublevel, i_azimuth, i_theta, i_phase] = local_filter(azimuth=azimuth, 
                                                                                   theta=theta, 
                                                                                   phase=phase, 
                                                                                   sf_0=sf_0, 
                                                                                   radius=radius)
    K = torch.Tensor(K)

    if verbose: print('K shape=', K.shape)
    if verbose: print('K min max=', K.min(), K.max())

    return K

### applying the filters OK

## 6. Inverse pyramid from the coefficients

In [ ]:
def inverse_pyramid(out, K_inv, N_X=N_X, N_Y=N_Y, 
                    n_sublevel=n_sublevel, n_azimuth = n_azimuth, n_theta = n_theta, n_phase=n_phase,
                    base_levels=base_levels, verbose=False):
    N_batch = out.shape[0]
    # width =  K_inv.shape[0]
    #n_levels = int(np.log(np.max((N_X, N_Y))/width)/np.log(base_levels)) + 1
    #n_sublevel, n_azimuth, n_theta, n_phase = K_inv.shape[2:]
    #print('n_levels =', n_levels)
    out__ = out.reshape((N_batch, n_levels, n_sublevel*n_azimuth*n_theta*n_phase))
    #K_ = K.reshape((width**2, n_sublevel*n_azimuth*n_theta*n_phase))
    #K_inv = torch.pinverse(K_)
    img_crop_rec =  torch.tensordot(out__, K_inv,  dims=1).reshape((N_batch, n_levels, width, width))

    img_rec = img_crop_rec[:, -1, :, :].unsqueeze(1)
    for i_level in range(n_levels-1)[::-1]: # from the top to the bottom of the pyramid
        img_rec = interpolate(img_rec, scale_factor=base_levels, mode=mode)
        h_res, w_res = img_rec.shape[-2:]
        img_rec[:, 0, (h_res//2-width//2):(h_res//2+width//2), (w_res//2-width//2):(w_res//2+width//2)] += img_crop_rec[:, i_level, :, :]
    img_rec = img_rec[:, :, (h_res//2-N_X//2):(h_res//2+N_X//2), (w_res//2-N_Y//2):(w_res//2+N_Y//2)]

    return img_rec

## 7. Retinal transformation function

In [ ]:
def pyr_transfo_ret(img_orig, K_azym_thet, n_azimuth = n_azimuth, n_theta = n_theta):
    img_crop = cropped_pyramid(torch.Tensor(img_orig[None, None, ...]), verbose=False)
    out = torch.tensordot(img_crop, K_azym_thet,  dims=2)
    K_ = K_azym_thet.reshape((width**2, n_sublevel*n_azimuth*n_theta*n_phase))
    K_azym_thet_inv = torch.pinverse(K_)
    img_rec = inverse_pyramid(out, K_azym_thet_inv, n_azimuth = n_azimuth, n_theta = n_theta)
    return img_rec
